In [2]:
import pandas as pd
pd.options.display.max_columns = 99

In [3]:
loans_2007 = pd.read_csv("loans_2007.csv")
loans_2007.head(5)

/dataquest/system/env/python3/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [4]:
thousand_rows = pd.read_csv("loans_2007.csv", nrows=1000)
thousand_rows.memory_usage(deep=True).sum()/(1024*1024)

1.5502090454101562

# Let's try tripling to 3000 rows and calculate the memory footprint for each chunk.

In [5]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
for chunk in chunk_iter:
    print(chunk.memory_usage(deep=True).sum()/(1024*1024))

4.649013519287109
4.6447601318359375
4.646517753601074
4.647870063781738
4.6440629959106445
4.6459455490112305
4.644536972045898
4.646905899047852
4.645031929016113
4.645082473754883
4.657794952392578
4.6566619873046875
4.663469314575195
4.896910667419434
0.8808088302612305


# How many rows in the data set?

In [6]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
total_rows = 0
for chunk in chunk_iter:
    total_rows += len(chunk)
print(total_rows)

42538


# How many columns have a numeric type? How many have a string type?

In [8]:
# Numeric columns
import numpy as np
loans_chunks = pd.read_csv('loans_2007.csv',chunksize=3000)

numeric = []
string = []
for lc in loans_chunks:
    nums = lc.select_dtypes(include=[np.number]).shape[1]
    numeric.append(nums)
    strs = lc.select_dtypes(include=['object']).shape[1]
    string.append(strs)

print(numeric)
print(string)

[31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 30, 30]
[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22]


In [22]:
chunks_iter = pd.read_csv("loans_2007.csv", chunksize=1000)
num_unique = {}
num_total = {}
for chunk in chunks_iter:
    for col in chunk.select_dtypes(include = ["object"]):
        num_unique_values = len(chunk[col].unique())
        num_total_values = len(chunk[col])
        if col in num_unique:
            num_unique[col] += num_unique_values
            num_total[col] += num_total_values
        else:
            num_unique[col] = num_unique_values
            num_total[col]  = num_total_values
        
print(num_unique)
print(num_total)
print("string columns contain values that are less than 50% unique\n")
for col in num_unique:
    if num_unique[col] / num_total[col] < 0.5:
        print(col)

{'earliest_cr_line': 12663, 'id': 1538, 'grade': 302, 'sub_grade': 1445, 'initial_list_status': 45, 'loan_status': 144, 'home_ownership': 144, 'pymnt_plan': 46, 'addr_state': 1811, 'int_rate': 2034, 'last_credit_pull_d': 2931, 'issue_d': 155, 'title': 29738, 'zip_code': 16201, 'verification_status': 128, 'application_type': 45, 'purpose': 573, 'last_pymnt_d': 2468, 'emp_title': 38477, 'term': 78, 'emp_length': 509, 'revol_util': 26289}
{'earliest_cr_line': 42538, 'id': 1538, 'grade': 42538, 'sub_grade': 42538, 'initial_list_status': 42538, 'loan_status': 42538, 'home_ownership': 42538, 'pymnt_plan': 42538, 'addr_state': 42538, 'int_rate': 42538, 'last_credit_pull_d': 42538, 'issue_d': 42538, 'title': 42538, 'zip_code': 42538, 'verification_status': 42538, 'application_type': 42538, 'purpose': 42538, 'last_pymnt_d': 42538, 'emp_title': 42538, 'term': 42538, 'emp_length': 42538, 'revol_util': 42538}
string columns contain values that are less than 50% unique

earliest_cr_line
grade
sub_g

# Which float columns have no missing values and could be candidates for conversion to the integer type?

In [23]:
loans_chunks = pd.read_csv('loans_2007.csv',chunksize=3000)

missing = []
for lc in loans_chunks:
    floats = lc.select_dtypes(include=['float'])
    missing.append(floats.apply(pd.isnull).sum())

combined_missing = pd.concat(missing)
combined_missing.groupby(combined_missing.index).sum().sort_values()

member_id                        3
total_rec_int                    3
total_pymnt_inv                  3
total_pymnt                      3
revol_bal                        3
recoveries                       3
policy_code                      3
out_prncp_inv                    3
out_prncp                        3
total_rec_late_fee               3
loan_amnt                        3
last_pymnt_amnt                  3
total_rec_prncp                  3
funded_amnt_inv                  3
funded_amnt                      3
dti                              3
collection_recovery_fee          3
installment                      3
annual_inc                       7
inq_last_6mths                  32
total_acc                       32
delinq_2yrs                     32
pub_rec                         32
delinq_amnt                     32
open_acc                        32
acc_now_delinq                  32
tax_liens                      108
collections_12_mths_ex_med     148
chargeoff_within_12_

# Calculate the total memory usage across all of the chunks.

In [24]:
loans_chunks = pd.read_csv('loans_2007.csv',chunksize=3000)

mem_usage = []

for lc in loans_chunks:
    mem_usage.append(lc.memory_usage(deep=True).sum() / 1024 ** 2)

sum(mem_usage)

66.2153730392456

# Determine which string columns you can convert to a numeric type if you clean them. For example, the int_rate column is only a string because of the % sign at the end.

In [28]:
convert_col_dtypes = {
    "sub_grade": "category", "home_ownership": "category", 
    "verification_status": "category", "purpose": "category"
}
def change_to_int(df, col_name):
    # Get the minimum and maximum values
    col_max = df[col_name].max()
    col_min = df[col_name].min()
    # Find the datatype
    for dtype_name in ['int8', 'int16', 'int32', 'int64']:
        # Check if this datatype can hold all values
        if col_max <  np.iinfo(dtype_name).max and col_min > np.iinfo(dtype_name).min:
            df[col_name] = df[col_name].astype(dtype_name)
            break

chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000, dtype=convert_col_dtypes, parse_dates=["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"])
mv_counts = {}
mem_usage = []
for chunk in chunk_iter:
    term_cleaned = chunk['term'].str.lstrip(" ").str.rstrip(" months")
    revol_cleaned = chunk['revol_util'].str.rstrip("%")
    chunk['term'] = pd.to_numeric(term_cleaned)
    chunk['revol_util'] = pd.to_numeric(revol_cleaned)
    chunk = chunk.dropna(how='all')
    float_cols = chunk.select_dtypes(include=['float'])
    for col in float_cols.columns:
        change_to_int(chunk,col)
        missing_values = len(chunk) - chunk[col].count()
        if col in mv_counts:
            mv_counts[col] = mv_counts[col] + missing_values
        else:
            mv_counts[col] = missing_values
    mem_usage.append(chunk.memory_usage(deep=True).sum() / 1024 ** 2)

sum(mem_usage)

ValueError: Cannot convert non-finite values (NA or inf) to integer